In [1]:
import os
os.getcwd()

'C:\\Users\\zzuge\\zzugest'

In [2]:
import pandas as pd
import numpy as np

loan_result = pd.read_csv('C:\\Users\\zzuge\\git\\bigcon2x\\bigcon22\\FD\\01_data\\loan_result.csv')
log_data = pd.read_csv('C:\\Users\\zzuge\\OneDrive\\문서\\user_event.csv')
user_spec = pd.read_csv('C:\\Users\\zzuge\\git\\bigcon2x\\bigcon22\\FD\\01_data\\user_spec.csv')

In [3]:
# application_n* 파생변수 생성
application_n = loan_result[['application_id']].value_counts()
application_n_df = pd.DataFrame(application_n) 
loan_result = pd.merge(loan_result, application_n_df, how = 'inner', on = ['application_id'])
loan_result.rename(columns = {0:'application_n'}, inplace = True)

In [4]:
# weekday* 파생변수 생성
# 0~6이 월~일을 의미한다
# 공휴일 : 6월1일, 6월 6일, 5월 5일, 3월 1일, 3월 9일
loan_result['loanapply_insert_time'] = pd.to_datetime(loan_result['loanapply_insert_time'])
loan_result['weekday'] = loan_result['loanapply_insert_time'].dt.dayofweek


In [5]:
loan_result['trash'] = loan_result['loanapply_insert_time'].dt.strftime('%m-%d')


In [6]:
holiday = ['06-01', '06-06', '05-05', '03-01', '03-09']
loan_result.loc[loan_result['trash'].isin(holiday), 'weekday'] = '6'
loan_result.drop(['trash', 'loanapply_insert_time'], axis = 1, inplace = True)


In [7]:
# age* 파생변수 생성
# 2022 - birth_year
user_spec[['age']] = 2022 - user_spec[['birth_year']]

In [8]:
user_spec['insert_time'] = pd.to_datetime(user_spec['insert_time']).dt.hour

In [9]:
first_join = pd.merge(loan_result, user_spec, left_on = 'application_id', right_on = 'application_id', how = 'left')
second_join = pd.merge(first_join, log_data, left_on = 'user_id', right_on = 'user_id', how = 'left')

In [10]:
loyalty_make = second_join.groupby('user_id')[['is_applied']].sum()
loyalty = loyalty_make['is_applied']
integrated = pd.merge(second_join, loyalty, left_on = 'user_id', right_on = 'user_id', how = 'inner')

In [11]:
integrated.rename(columns={'is_applied_x':'is_applied', 'is_applied_y':'loyalty'}, inplace = True)
integrated[['user_id', 'loyalty']] = integrated[['user_id', 'loyalty']].astype('int64')

In [12]:

integrated.drop('birth_year', axis = 1, inplace = True)

In [13]:
integrated.to_csv('C:\\Users\\zzuge\\OneDrive\\문서\\integrated1.csv')

In [19]:
df = pd.read_csv('C:\\Users\\zzuge\\OneDrive\\문서\\integrated.csv')
df

,Unnamed: 0,application_id,bank_id,product_id,loan_limit,loan_rate,is_applied,application_n,weekday,user_id,...,Login,ViewLoanApplyIntro,StartLoanApply,CompleteIDCertification,EndLoanApply,UseLoanManage,UsePrepayCalc,UseDSRCalc,GetCreditInfo,loyalty
0,0,1748340,7,191,42000000.0,13.6,NaN,35,1,430982,...,0.0,16.0,50.0,15.0,50.0,1.0,0.0,0.0,2.0,9
1,1,1748340,25,169,24000000.0,17.9,NaN,35,1,430982,...,0.0,16.0,50.0,15.0,50.0,1.0,0.0,0.0,2.0,9
2,2,1748340,2,7,24000000.0,18.5,NaN,35,1,430982,...,0.0,16.0,50.0,15.0,50.0,1.0,0.0,0.0,2.0,9
3,3,1748340,4,268,29000000.0,10.8,NaN,35,1,430982,...,0.0,16.0,50.0,15.0,50.0,1.0,0.0,0.0,2.0,9
4,4,1748340,11,118,5000000.0,16.4,NaN,35,1,430982,...,0.0,16.0,50.0,15.0,50.0,1.0,0.0,0.0,2.0,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13527245,13527245,1513515,24,263,5000000.0,15.4,NaN,14,4,31658,...,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0
13527246,13527246,1513515,55,24,19000000.0,11.5,NaN,14,4,31658,...,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0
13527247,13527247,1513515,15,204,16000000.0,14.3,NaN,14,4,31658,...,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0
13527248,13527248,1513515,30,85,10000000.0,14.3,NaN,14,4,31658,...,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0


In [23]:
df[df['is_applied'].isnull()].isnull().sum()

Unnamed: 0                                   0
application_id                               0
bank_id                                      0
product_id                                   0
loan_limit                                1757
loan_rate                                 1757
is_applied                             3257239
application_n                                0
weekday                                      0
user_id                                      0
gender                                   36470
insert_time                                  0
credit_score                            265464
yearly_income                                6
income_type                                  0
company_enter_month                      96769
employment_type                              0
houseown_type                                0
desired_amount                               0
purpose                                      0
personal_rehabilitation_yn               15472
personal_reha